In [ ]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, ShuffleSplit, GridSearchCV 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
import seaborn as sns 
import collections
from itertools import zip_longest

# 下2個のモジュールはlabelencodingを全体に適用させるためのもの
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
train_data = pd.read_csv('../input/contradictory-my-dear-watson/train.csv')
train_matrix = train_data.shape[0]

In [ ]:
train_data

In [ ]:
test_data = pd.read_csv('../input/contradictory-my-dear-watson/test.csv')
test_data.shape

In [ ]:
data = pd.concat([train_data, test_data], axis=0)

In [ ]:
data

In [ ]:
data.drop(['lang_abv'],1, inplace=True)

In [ ]:
# separate sentences

In [ ]:
def pre_separate(self):
    num_need_columns = []
    columns_element = collections.OrderedDict()    # 順序を保持する
    for content in self['premise']:
        num_need_columns.append(len(content.split(' ')))
    need_num = max(num_need_columns)
    
    for num in range(need_num):
        columns_element['p_' + '{}'.format(num)] = []
        
    for content in self['premise']:
        for n, word in zip_longest(range(need_num), content.split(' ')):   #zip:多い分の要素が無視される,  zip_longest:足りない分の要素が埋められる
            try:   # 変数値の存在確認
                columns_element['p_' + '{}'.format(n)].append(word)
            except:
                columns_element['p_' + '{}'.format(n)].append(None)
    
    for n in range(need_num):
        self['p_' + '{}'.format(n)] = columns_element['p_' + '{}'.format(n)]
        
    return self

In [ ]:
data = pre_separate(data)

In [ ]:
c=collections.OrderedDict()
c['P_0']=[]
c['P_0'].append(2)
c

In [ ]:
data

In [ ]:
def hy_separate(self):
    num_need_columns = []
    columns_element = collections.OrderedDict()    # 順序を保持する
    for content in self['hypothesis']:
        num_need_columns.append(len(content.split(' ')))
    need_num = max(num_need_columns)
    
    for num in range(need_num):
        columns_element['h_' + '{}'.format(num)] = []
        
    for content in self['hypothesis']:
        for n, word in zip_longest(range(need_num), content.split(' ')):   #zip:多い分の要素が無視される,  zip_longest:足りない分の要素が埋められる
            try:   
                columns_element['h_' + '{}'.format(n)].append(word)
            except:
                columns_element['h_' + '{}'.format(n)].append(None)
    
    for n in range(need_num):
        self['h_' + '{}'.format(n)] = columns_element['h_' + '{}'.format(n)]
        
    return self

In [ ]:
data = hy_separate(data) 

In [ ]:
data

In [ ]:
sep_data = data.copy()

In [ ]:
#create dummies variables

In [ ]:
outputs_data = sep_data['label']
sep_data.drop(['id', 'premise', 'hypothesis', 'label'],1, inplace=True)

In [ ]:
le =  LabelEncoder()

In [ ]:
sep_data = sep_data.apply(le.fit_transform)

In [ ]:
inputs = sep_data.copy()

In [ ]:
def trans_int(self):
    for column in self.columns:
        try:
            self[column] = self[column].astype(int)
        except:
            continue     # 繰り返しに戻る
            
        return self

In [ ]:
trans_int(inputs)

In [ ]:
inputs

In [ ]:
inputs_train = inputs[:train_matrix]
inputs_test = inputs[train_matrix:]

In [ ]:
outputs_train = outputs_data[:train_matrix]

In [ ]:
outputs_train

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(inputs_train, outputs_train, test_size=0.85)

In [ ]:
gnb = GaussianNB().fit(x_train, y_train)
knnc = KNeighborsClassifier().fit(x_train, y_train)
cartc = DecisionTreeClassifier(random_state=42).fit(x_train, y_train)
rfc = RandomForestClassifier(random_state=42, verbose=False).fit(x_train, y_train)
gbmc = GradientBoostingClassifier(verbose=False).fit(x_train, y_train)
xgbc = XGBClassifier().fit(x_train, y_train)
lgbmc = LGBMClassifier().fit(x_train, y_train)
catbc = CatBoostClassifier(verbose=False).fit(x_train, y_train)

In [ ]:
modelsc = [gnb, knnc, cartc, rfc, gbmc, lgbmc, catbc]

In [ ]:
"""
for model in modelsc:
    name = model.__class__.__name__
    predict = model.predict(x_val)
    R2CV = cross_val_score(model, x_val, y_val, cv=10, verbose=False).mean()
    error = -cross_val_score(model, x_val, y_val, cv=10, scoring='neg_mean_squared_error', verbose=False).mean()
    print(name+':')
    print('-------'*10)
    print(accuracy_score(y_val, predict))
    print('-------'*10)
    print(R2CV)
    print(np.sqrt(error))
    print('-'*30)
"""

In [ ]:
# 上記の結果より、勾配ブースティング分類法を用いる
gbmc2 = GradientBoostingClassifier(verbose=False).fit(inputs_train, outputs_train)

In [ ]:
pre_test = gbmc2.predict(inputs_test).astype(int)

In [ ]:
# create submission
sub_df = pd.read_csv('../input/contradictory-my-dear-watson/sample_submission.csv')
sub_df['prediction'] = pre_test
sub_df.to_csv('submission.csv', index=False)

In [ ]:
pre_test.astype(int)

In [ ]:
sub_df